# Import locomotion and IGL

In [1]:
import sys
import os
PATH_TO_DIRECTORY = os.getcwd()
sys.path.append(PATH_TO_DIRECTORY)
import locomotion

from math import ceil, exp, log, sin, asin, pi, acosh, cosh, sinh, cos, acos, atanh, tanh
from numpy import min, mean, std, array, linalg, dot, cross
from scipy.optimize import minimize_scalar

In [2]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact

# Testing out IGL methods

In [44]:
# A cube, just trying out stuff
v = np.array([[-0.5, -0.5, 0.5],
              [0.5, -0.5, 0.5],
              [-0.5, 0.5 , 0.5],
              [0.5, 0.5, 0.5],
              [-0.5, 0.5, -0.5],
              [0.5, 0.5, -0.5],
              [-0.5, -0.5, -0.5],
              [0.5, -0.5, -0.5]])

f = np.array([[0, 1, 2],
              [1, 2, 3],
              [2, 3, 5],
              [2, 5, 4],
              [0, 2, 6],
              [0, 2, 4],
              [0, 1, 7],
              [0, 7, 6],
              [6, 7, 4],
              [7, 4, 5],
              [1, 3, 5],
              [1, 5, 7]])

In [45]:
v, f = igl.read_triangle_mesh("Med_NSS_07.delaunay.off")
# v: Array of vertices
# f: Array of vertices making up each face
v

array([[100.,  80.,  99.],
       [110.,  80.,  99.],
       [ 90.,  80.,  99.],
       ...,
       [190.,  70.,  50.],
       [190.,  80.,  50.],
       [190.,  90.,  50.]])

In [46]:
## Find the (longest ordered) open boundary - array indices of each vertex
bnd = igl.boundary_loop(f)

array([ 29,  40,  47,  36,  71,  97,  65,  83, 115, 179, 215, 282, 273,
       272, 271, 270, 269, 268, 267, 266, 265, 264, 274, 283, 290, 296,
       303, 308, 309, 311, 313, 315, 317, 318, 320, 321, 323, 325, 332,
       342, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 351, 341,
       230, 149, 147, 109,  78,  57,  35], dtype=int64)

In [69]:
## Map the boundary to a circle, preserving edge proportions
# Array of 2D coordinates, vertices of unit circle
bnd_uv = igl.map_vertices_to_circle(v, bnd)

In [70]:
p = plot(bnd_uv, f, uv=uv, shading={"wireframe": False, "flat": False}, return_plot=True)

@interact(mode=['3D','2D'])
def switch(mode):
    if mode == "3D":
        # Original faces and vertices
        plot(v, f, uv=uv, shading={"wireframe": False, "flat": False}, plot=p)
    if mode == "2D":
        # Parametrised vertices
        plot(bnd_uv, bnd_f, uv=uv, shading={"wireframe": True, "flat": False}, plot=p)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.7553567…

interactive(children=(Dropdown(description='mode', options=('3D', '2D'), value='3D'), Output()), _dom_classes=…

In [74]:
## Harmonic parametrization for the internal vertices
# All vertices mapped to unit circle given boundary circle

uv = igl.harmonic_weights(v, f, bnd, bnd_uv, 1)
# Make vertices 3D with 0 for plotting
v_p = np.hstack([uv, np.zeros((uv.shape[0],1))])

In [75]:
p = plot(v, f, uv=uv, shading={"wireframe": False, "flat": False}, return_plot=True)

@interact(mode=['3D','2D'])
def switch(mode):
    if mode == "3D":
        # Original faces and vertices
        plot(v, f, uv=uv, shading={"wireframe": False, "flat": False}, plot=p)
    if mode == "2D":
        # Parametrised vertices
        plot(v_p, f, uv=uv, shading={"wireframe": True, "flat": False}, plot=p)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(95.0, 45.…

interactive(children=(Dropdown(description='mode', options=('3D', '2D'), value='3D'), Output()), _dom_classes=…

# Get an animal as an example

In [3]:
info_file = PATH_TO_DIRECTORY + "/samples/sample_check.json"
animals = locomotion.getAnimalObjs( info_file )

LOG: Extracting coordinates for Animal SS_01...
LOG: Extracting coordinates for Animal SS_02...
LOG: Extracting coordinates for Animal NSS_01...
LOG: Extracting coordinates for Animal NSS_02...


In [4]:
# This is Animal SS_01
a = animals[0]

# Modifying getFlatCoordinates

In [13]:
PERTURBATION = 0.000000001
TOLERANCE = 0.00001
a.setPerturbation(PERTURBATION)
a.setTolerance(TOLERANCE)
a.setGridSize(10)

In [14]:
frequencies = locomotion.heatmap.getFrequencies(a, a.getExpStartTime(), a.getExpEndTime())
original_coordinates = locomotion.heatmap.getVertexCoordinates(a, frequencies)
a.setNumVerts(len(original_coordinates))
a.setRegularCoordinates(original_coordinates)
triangles = locomotion.heatmap.getTriangles(a)
a.setTriangulation(triangles)

#calculate and store colors for output file
colors = locomotion.heatmap.getColors(a)
a.setColors(colors)

v, f = original_coordinates, triangles

In [15]:
# insert IGL stuff here!!!
# Check that boundary vertices do the same thing
bnd = igl.boundary_loop(f)
# Check why the boundary from igl and from our own function is different?
# bnd = locomotion.heatmap.getBoundaryVertices(a)
# bnd

## Map the boundary to a circle, preserving edge proportions
# Array of 2D coordinates, vertices of unit circle
bnd_uv = igl.map_vertices_to_circle(v, bnd)

## Harmonic parametrization for the internal vertices
# All vertices mapped to unit circle given boundary circle

uv = igl.harmonic_weights(v, f, bnd, bnd_uv, 1)
uv

flat_coordinates = [[tanh(c[0])*cos(c[1]),tanh(c[0])*sin(c[1])] for c in uv]

TypeError: boundary_loop(): incompatible function arguments. The following argument types are supported:
    1. (f: array) -> object

Invoked with: [[0, 10, 11], [0, 11, 1], [1, 11, 12], [1, 12, 2], [2, 12, 13], [2, 13, 3], [3, 13, 14], [3, 14, 4], [4, 14, 15], [4, 15, 5], [5, 15, 16], [5, 16, 6], [6, 16, 17], [6, 17, 7], [7, 17, 18], [7, 18, 8], [8, 18, 19], [8, 19, 9], [10, 20, 21], [10, 21, 11], [11, 21, 22], [11, 22, 12], [12, 22, 23], [12, 23, 13], [13, 23, 24], [13, 24, 14], [14, 24, 25], [14, 25, 15], [15, 25, 26], [15, 26, 16], [16, 26, 27], [16, 27, 17], [17, 27, 28], [17, 28, 18], [18, 28, 29], [18, 29, 19], [20, 30, 31], [20, 31, 21], [21, 31, 32], [21, 32, 22], [22, 32, 33], [22, 33, 23], [23, 33, 34], [23, 34, 24], [24, 34, 35], [24, 35, 25], [25, 35, 36], [25, 36, 26], [26, 36, 37], [26, 37, 27], [27, 37, 38], [27, 38, 28], [28, 38, 39], [28, 39, 29], [30, 40, 41], [30, 41, 31], [31, 41, 42], [31, 42, 32], [32, 42, 43], [32, 43, 33], [33, 43, 44], [33, 44, 34], [34, 44, 4

In [134]:
p = plot(v, f, uv=uv, shading={"wireframe": False, "flat": False}, return_plot=True)

@interact(mode=['3D','2D'])
def switch(mode):
    if mode == "3D":
        # Original faces and vertices
        plot(v, f, uv=uv, shading={"wireframe": False, "flat": False}, plot=p)
    if mode == "2D":
        # Parametrised vertices
        plot(v_p, f, uv=uv, shading={"wireframe": True, "flat": False}, plot=p)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(95.0, 45.…

interactive(children=(Dropdown(description='mode', options=('3D', '2D'), value='3D'), Output()), _dom_classes=…

In [155]:
f = a.getTriangulation()
# boundary_vertices = igl.boundary_loop(f)
type(a.getRegularCoordinates())
a.getRegularCoordinates()

[[0, 0, 43.17836348553121],
 [0, 10, 43.17836348553121],
 [0, 20, 43.17836348553121],
 [0, 30, 43.17836348553121],
 [0, 40, 43.17836348553121],
 [0, 50, 43.17836348553121],
 [0, 60, 43.17836348553121],
 [0, 70, 43.17836348553121],
 [0, 80, 43.17836348553121],
 [0, 90, 43.17836348553121],
 [10, 0, 43.17836348553121],
 [10, 10, 43.17836348553121],
 [10, 20, 43.17836348553121],
 [10, 30, 43.17836348553121],
 [10, 40, 43.17836348553121],
 [10, 50, 43.17836348553121],
 [10, 60, 43.17836348553121],
 [10, 70, 43.17836348553121],
 [10, 80, 43.17836348553121],
 [10, 90, 43.17836348553121],
 [20, 0, 43.17836348553121],
 [20, 10, 43.17836348553121],
 [20, 20, 43.17836348553121],
 [20, 30, 43.17836348553121],
 [20, 40, 43.17836348553121],
 [20, 50, 43.17836348553121],
 [20, 60, 43.17836348553121],
 [20, 70, 43.17836348553121],
 [20, 80, 43.17836348553121],
 [20, 90, 43.17836348553121],
 [30, 0, 43.17836348553121],
 [30, 10, 43.17836348553121],
 [30, 20, 43.17836348553121],
 [30, 30, 43.17836348553

In [17]:
grid_size = 10
# locomotion.trajectory.getCurveData( a )
# locomotion.heatmap.getSurfaceData(a, grid_size)
flat = locomotion.heatmap.getFlatCoordinates(a)

[[  0.           0.          43.17836349]
 [  0.          10.          43.17836349]
 [  0.          20.          43.17836349]
 [  0.          30.          43.17836349]
 [  0.          40.          43.17836349]
 [  0.          50.          43.17836349]
 [  0.          60.          43.17836349]
 [  0.          70.          43.17836349]
 [  0.          80.          43.17836349]
 [  0.          90.          43.17836349]
 [ 10.           0.          43.17836349]
 [ 10.          10.          43.17836349]
 [ 10.          20.          43.17836349]
 [ 10.          30.          43.17836349]
 [ 10.          40.          43.17836349]
 [ 10.          50.          43.17836349]
 [ 10.          60.          43.17836349]
 [ 10.          70.          43.17836349]
 [ 10.          80.          43.17836349]
 [ 10.          90.          43.17836349]
 [ 20.           0.          43.17836349]
 [ 20.          10.          43.17836349]
 [ 20.          20.          43.17836349]
 [ 20.          30.          43.17

In [19]:
flat

[[-0.43411564284552123, 0.006390858047571407, 0],
 [-0.42965115608994237, 0.05244618607913534, 0],
 [-0.4169552155194943, 0.09638912341686368, 0],
 [-0.3967034835017728, 0.13630691545409263, 0],
 [-0.36993414714968925, 0.17053471235121184, 0],
 [-0.3379518385611276, 0.1977412210832073, 0],
 [-0.3022125295639516, 0.21698150610383055, 0],
 [-0.26420255089875916, 0.2277156857455989, 0],
 [-0.22532393070568899, 0.22979695905437114, 0],
 [-0.18679585663806797, 0.2234357836562123, 0],
 [-0.4301005308804927, -0.03972827410576936, 0],
 [-0.4160329151862911, 0.012663915358168276, 0],
 [-0.39655245143287965, 0.060582396203986935, 0],
 [-0.3715354774847028, 0.10313434491219779, 0],
 [-0.34142071422451387, 0.13945639390174286, 0],
 [-0.3069460186782772, 0.16868215534922473, 0],
 [-0.26918458774448134, 0.19014897977640974, 0],
 [-0.2294432837629942, 0.20362435714001093, 0],
 [-0.18911674456460673, 0.20949742435295338, 0],
 [-0.14957902495887346, 0.20914873543619467, 0],
 [-0.41781033423375236, -0.0